# Tutorial: Spa Single-Track Sensitivity

This notebook applies the high-level Sensitivity API to Spa-Francorchamps.
The setup uses `run_lap_sensitivity_study(...)` with the torch backend (autodiff default).

## Parameter and objective setup

Parameters:

1. Vehicle mass (`vehicle.mass`)
2. CoG height (`vehicle.cg_height`)
3. Yaw inertia (`vehicle.yaw_inertia`)
4. Drag coefficient (`vehicle.drag_coefficient`)

Objectives:

- `lap_time_s`
- `energy_kwh`

In [ ]:
from __future__ import annotations

from pathlib import Path
import sys

import matplotlib.pyplot as plt
import pandas as pd


def find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "src").exists():
            return candidate
    raise RuntimeError("Could not locate repository root from current working directory")


repo_root = find_repo_root(Path.cwd())
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

examples_sensitivity = repo_root / "examples" / "sensitivity"
if str(examples_sensitivity) not in sys.path:
    sys.path.insert(0, str(examples_sensitivity))

from apexsim.analysis import (
    SensitivityStudyParameter,
    build_sensitivity_study_model,
    run_lap_sensitivity_study,
)
from apexsim.simulation import build_simulation_config
from apexsim.tire import default_axle_tire_parameters
from apexsim.track import load_track_csv
from apexsim.vehicle import SingleTrackPhysics, build_single_track_model
from common import example_vehicle_parameters, sensitivity_output_root, spa_track_path

pd.set_option("display.max_columns", 30)
pd.set_option("display.width", 160)

In [ ]:
track = load_track_csv(spa_track_path())
vehicle = example_vehicle_parameters()
tires = default_axle_tire_parameters()
physics = SingleTrackPhysics()

simulation_config = build_simulation_config(
    compute_backend="torch",
    torch_device="cpu",
    torch_compile=False,
    max_speed=115.0,
)

study_model = build_sensitivity_study_model(
    model_factory=build_single_track_model,
    model_inputs={
        "vehicle": vehicle,
        "tires": tires,
        "physics": physics,
    },
    label="Spa-Francorchamps",
)

study_result = run_lap_sensitivity_study(
    track=track,
    study_model=study_model,
    simulation_config=simulation_config,
    parameters=[
        SensitivityStudyParameter(name="mass", target="vehicle.mass", label="Vehicle mass"),
        SensitivityStudyParameter(name="cg_height", target="vehicle.cg_height", label="Center of gravity height"),
        SensitivityStudyParameter(name="yaw_inertia", target="vehicle.yaw_inertia", label="Yaw inertia"),
        SensitivityStudyParameter(name="drag_coefficient", target="vehicle.drag_coefficient", label="Drag coefficient"),
    ],
)

long_df = study_result.to_dataframe().sort_values(["objective", "parameter"], kind="stable")
pivot_df = study_result.to_pivot().sort_index(kind="stable")

long_df[[
    "objective",
    "parameter_label",
    "objective_value",
    "sensitivity_raw",
    "sensitivity_pct_per_pct",
]]

In [ ]:
output_dir = sensitivity_output_root() / "spa_single_track"
output_dir.mkdir(parents=True, exist_ok=True)

long_df.to_csv(output_dir / "sensitivities_long.csv", index=False)
pivot_df.to_csv(output_dir / "sensitivities_pivot.csv")

plot_df = long_df[["parameter_label", "objective", "sensitivity_pct_per_pct"]].pivot(
    index="parameter_label",
    columns="objective",
    values="sensitivity_pct_per_pct",
)

fig, axes = plt.subplots(1, 2, figsize=(12.0, 4.5), constrained_layout=True)
plot_df["lap_time_s"].plot(kind="bar", ax=axes[0], color="#1565c0", title="Lap-time sensitivity")
plot_df["energy_kwh"].plot(kind="bar", ax=axes[1], color="#2e7d32", title="Energy sensitivity")
axes[0].set_ylabel("Relative output change / relative input change")
axes[1].set_ylabel("Relative output change / relative input change")
for axis in axes:
    axis.tick_params(axis="x", rotation=20)
    axis.grid(alpha=0.25, axis="y")

plot_path = output_dir / "sensitivity_bars.png"
fig.savefig(plot_path, dpi=160)
plt.close(fig)

print(f"Artifacts written to: {output_dir}")
pivot_df

## Interpretation

Compare `%/%` sensitivities objective-wise:

1. Rank parameters for `lap_time_s`
2. Rank parameters for `energy_kwh`
3. Compare ranking shifts and relate them to Spa sectors (high-speed vs. corner-dominated)

For finite-difference regression checks, pass `runtime=SensitivityRuntime(method="finite_difference")` into `run_lap_sensitivity_study(...)`.